<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from __future__ import absolute_import, division, print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    path='imdb.npz',
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

In [3]:
unique, counts = np.unique(y_train, return_counts=True)
positive_sample_count = counts[1]

Following is all the  already done part from milestone 2

In [4]:
idx = np.argwhere(y_train>0) # Select positive comment's index in training data
np.random.seed(seed=100) # use seed to ensure selected records are always same
np.random.shuffle(idx) #Shuffle it at random
FRAC = 0.25
idxs = idx[:int(len(idx)*FRAC)]
y_trains = y_train[idxs]
x_trains = x_train[idxs]

# preserve negative cases
idxn = np.argwhere(y_train==0)
x_train0 = x_train[idxn]
y_train0 = y_train[idxn]

temp_idx = np.arange(0, len(idxs), 1)
temp_idx_multi = np.tile(temp_idx, (1, 10)).T # shape is (31250, 1)
np.random.seed(seed=200) # rerun from temp_idx_multi definition cell to ensure same result
np.random.shuffle(temp_idx_multi)
sample_idx = temp_idx_multi[:positive_sample_count].squeeze(1)
x_train[sample_idx] # no point of using just the positive, since we are going by indeces anyway

array([list([1, 12, 16, 576, 364, 352, 12, 49, 139, 12, 272, 40, 36, 4148, 19, 6, 344, 374, 14996, 190, 12, 127, 28, 6, 52, 114, 347, 5, 94, 776, 8, 794, 709, 153, 103, 1205, 41, 3375, 4595, 1491, 336, 4431, 9, 626, 39, 4, 9368, 3689, 19, 4, 339, 7, 41, 1042, 37, 59, 9, 4549, 260, 35, 1586, 19, 4, 1042, 630, 56, 2753, 41, 6, 313, 5, 2644, 41, 6, 519, 21, 746, 4, 86, 891, 234, 7, 4, 20, 4431, 1098, 90, 5, 18274, 90, 11, 41, 9127, 95, 59, 24965, 41, 3608, 745, 732, 523, 23, 90, 5, 3003, 33, 164, 366, 59, 47, 90, 59, 1421, 1356, 21, 103, 160, 3608, 6244, 659, 46, 15, 4431, 556, 15, 1042, 5, 3367, 8, 516, 4865, 322, 6555, 14, 9, 103, 4431, 5, 90, 79, 12, 23, 11, 4, 893, 11394, 95, 59, 3003, 116, 38, 1356, 5, 1098, 6244, 5, 303, 23, 1027, 8, 516, 745, 937, 59, 1505, 1388, 5, 2263, 18, 5, 490, 28, 8, 851, 4, 20, 8, 169, 46, 48, 4431, 1442, 42, 24, 444, 52, 20, 1791, 72, 6, 176, 7, 61, 113, 25, 124, 4, 226, 1454, 18, 4, 3608, 5, 5564, 33, 164, 366, 25, 28, 90, 173]),
       list([1, 13, 81, 2

And now we process the data to have all the items (reviews) having the same number of tokens (words).

In [5]:
x_trains_os = x_train[sample_idx] # again, we are using indeces
y_trains_os = y_train[sample_idx]
x_train_assembled = np.concatenate((x_train0, x_trains_os), axis = None)
y_train_assembled = np.concatenate((y_train0, y_trains_os), axis = None)
shuffled_idx = np.arange(0, len(y_train_assembled), 1)
np.random.seed(seed=300)
np.random.shuffle(shuffled_idx)
print(shuffled_idx.shape)

(25000,)


In [6]:
shuffled_idx

array([14841, 22400, 10026, ...,  5834,  6625, 17617])

In [7]:
x_train_assembled_shuffled = x_train_assembled[shuffled_idx]
y_train_assembled_shuffled = y_train_assembled[shuffled_idx]
print(x_train_assembled_shuffled.shape, y_train_assembled_shuffled.shape)

(25000,) (25000,)


In [8]:
word_index = tf.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

index_word = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(encoded_array):
    return ' '.join([index_word.get(i, '?') for i in encoded_array])

In [9]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(x_train_assembled_shuffled,
                                                        value=word_index["<PAD>"],
                                                        padding='pre',
                                                        maxlen=256)

test_data = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_index["<PAD>"],
                                                       padding='pre',
                                                       maxlen=256)

In [10]:
train_data.shape

(25000, 256)

In [11]:
test_data.shape

(25000, 256)